In [4]:
import pandas as pd

# Original data
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'Alice', 'David', 'Bob'],
    'Gender': ['Female', 'Male', 'Male', 'Female', 'Male', 'Male'],
    'Age': [25, 32, 28, 25, 25, 32],
    'ZIP': [12345, 54321, 67890, 12345, 12345, 54321],
    'Diagnosis': ['Flu', 'Allergy', 'Headache', 'Flu', 'Sore Throat', 'Allergy']
}

# Create the original dataframe
df_original = pd.DataFrame(data)

# Function to perform generalization for Age and ZIP
def generalize_quasi_identifiers(df):
    df['Age'] = df['Age'] // 10 * 10  # Generalize Age to the nearest 10
    df['ZIP'] = df['ZIP'] // 1000 * 1000  # Generalize ZIP to the nearest 1000
    return df

# Perform generalization on quasi-identifiers
df_anonymous = generalize_quasi_identifiers(df_original.copy())

# Display the 3-anonymous table
print("Original Table:")
print(df_original)
print("\n3-Anonymous Table:")
print(df_anonymous)


Original Table:
      Name  Gender  Age    ZIP    Diagnosis
0    Alice  Female   25  12345          Flu
1      Bob    Male   32  54321      Allergy
2  Charlie    Male   28  67890     Headache
3    Alice  Female   25  12345          Flu
4    David    Male   25  12345  Sore Throat
5      Bob    Male   32  54321      Allergy

3-Anonymous Table:
      Name  Gender  Age    ZIP    Diagnosis
0    Alice  Female   20  12000          Flu
1      Bob    Male   30  54000      Allergy
2  Charlie    Male   20  67000     Headache
3    Alice  Female   20  12000          Flu
4    David    Male   20  12000  Sore Throat
5      Bob    Male   30  54000      Allergy


In [3]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/EXAM/dictionary.txt'
import hashlib
import itertools
from itertools import product

def hash_break(password, hash_order):
    for func in hash_order:
        password = func(password.encode()).hexdigest()
    return password

hash_functions = [hashlib.md5, hashlib.sha1, hashlib.sha224, hashlib.sha256, hashlib.sha384, hashlib.sha512]

h = '1140ccd48b9dfa0709afe843add51b7b109b6c6cae918854bcc55fb77ebb04005c0d38cdcc7b0c2882aa4e0123051ad9b59fc1894ba853517529e1984fe51292'
cnt = 0

with open(file_path, 'r') as file:
    for line in file:
        if cnt != 0:
            break
        pwd = line.strip()
        password = pwd + pwd[::-1]
        for hash_order in product(hash_functions, repeat=4):
            hashed = hash_break(password, hash_order)
            if hashed == h:
                print(pwd)
                cnt+=1
                break

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
madam


In [ ]:
#panda append is no longer supported in some version of python, we recommend running our files in google colab
!pip install adversarial-robustness-toolbox
!pip hash
!pip install datasketch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 12.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.15.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.1.3 which is incompatible.

Usage:   pip3 hash [options] <file> ...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.ensemble import VotingClassifier
import random
import hashlib
from sklearn.preprocessing import MinMaxScaler
from art.attacks.inference.membership_inference import MembershipInferenceBlackBoxRuleBased
from art.estimators.classification import SklearnClassifier
import time
start_time = time.time()

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CIS-545_Group2FinalProject/WorkingFiles/bank-full.csv', encoding = 'utf8', delimiter=';') #change drive location to file location
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
#Convert string columns to categorical
df[['job', 'marital','education','default', 'housing', 'loan', 'contact', 'month', 'poutcome','y']] = df[['job', 'marital','education','default', 'housing', 'loan', 'contact', 'month', 'poutcome','y']].astype('category')

In [ ]:
#Get dummies for categorical columns
df = pd.concat((df,pd.get_dummies(df[['job', 'marital','education','default', 'housing', 'loan', 'contact', 'month', 'poutcome','y']], drop_first=True)),axis=1)

In [ ]:
#X and y dataframes
X = df.drop(columns=['job', 'marital','education','default', 'housing', 'loan', 'contact', 'month', 'poutcome','y','y_yes']) #drop categorical columns
y = df['y_yes']

In [ ]:
# Min-Max Normalization
df_cont = X.copy()
#Remove all columns between column index 8 to 42
df_cont.drop(df_cont.iloc[:, 7:], inplace=True, axis=1)

df_disc = X.copy()
#Remove all columns between column index 8 to 42
df_disc.drop(df_disc.iloc[:, :7], inplace=True, axis=1)

df_cont = (df_cont-df_cont.min())/(df_cont.max()-df_cont.min())
X_scaled= pd.concat((df_cont, df_disc), 1)


In [ ]:
#add constant to predictor variables
X_scaled = sm.add_constant(X_scaled)

In [ ]:
X_attack = X_scaled.copy()

In [ ]:
# split data into training and test sets: scikit-learn
X_train, X_test, y_train, y_test = train_test_split(X_scaled.iloc[:, 0:43], y, test_size=0.20, random_state = 42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(36168, 43)
(36168,)
(9043, 43)
(9043,)


In [ ]:
# Construct a logistic regression model
def logistic_regression(x,y):
  logistic_model = LogisticRegression(class_weight='balanced', solver='newton-cg', penalty='l2', max_iter=1000, random_state=42)
  logistic_model = logistic_model.fit(x,y)

  coeffs = logistic_model.coef_[0,:]     # Coefficients as an array for np.dot

  return logistic_model, coeffs

def prediction(x,coeffs):
    y_pred = np.dot(x,coeffs)            # Prediction for each X
    p_pred = 1/(1+np.exp(-y_pred))             # Probability of Y = 1

    return y_pred,p_pred

def evaluation(y,p_pred):
  from sklearn import metrics
  fpr, tpr, thresholds = metrics.roc_curve(y, p_pred);

  threshold = 0.75                                                # Default threshold = 0.5
  Y_pred = pd.DataFrame() # Declare a panda
  Y_pred['Class_true'] = y
  Y_pred['Class_pred'] = 1*(p_pred>threshold)
  Y_pred['P_pred'] = p_pred
  Y_pred['TP'] = 1*((Y_pred['Class_true']==1) & (Y_pred['Class_pred'] == 1))
  Y_pred['FP'] = 1*((Y_pred['Class_true']==0) & (Y_pred['Class_pred'] == 1))
  Y_pred['TN'] = 1*((Y_pred['Class_true']==0) & (Y_pred['Class_pred'] == 0))
  Y_pred['FN'] = 1*((Y_pred['Class_true']==1) & (Y_pred['Class_pred'] == 0))
  Y_pred

  print('#################################')
  print('Accuracy = ', metrics.accuracy_score(Y_pred['Class_true'],Y_pred['Class_pred']))
  print('#################################')
  return Y_pred


In [ ]:
#Get Logistic Model and Summary
model,coeffs = logistic_regression(X_train,y_train)

In [ ]:
#How good is the model with train data?
y_pred,p_pred = prediction(X_train,coeffs)
Y_pred = evaluation(y_train,p_pred)

#################################
Accuracy =  0.8706591462065915
#################################


In [ ]:
#Test Results
y_pred,p_pred = prediction(X_test,coeffs)
Y_pred = evaluation(y_test,p_pred)

#################################
Accuracy =  0.8690699988941722
#################################


In [ ]:
# Create a training data frame to split in n disjoint subsets
df_train = X_train.copy()
df_train['y_yes'] = y_train.copy()

In [ ]:
#split in n disjoint subsets
n = 6
size = int((len(df_train) - len(df_train)%n)/n) #removing the remainder is discarding some data
#print(len(df_train))
sample_ = {}
df_holder = df_train

for i in range(1, n + 1):
    sample_[i] = df_holder.sample(size)
    df_holder = df_holder.drop(sample_[i].index)

In [ ]:
#Build Models for ensemble
X_sample_={}
y_sample_={}
y_pred_={}
p_pred_={}
Y_pred_={}
model_={}
coeffs_={}


#X and y dataframes
for i in range(1,n+1,1):
  print(i)
  X_sample_[i] = sample_[i].drop(columns=['y_yes']) #drop y column
  y_sample_[i] = sample_[i]['y_yes']
  #print(X_sample_[i])
  #Get Logistic Model and Summary
  model_[i],coeffs_[i] = logistic_regression(X_sample_[i],y_sample_[i])
  #How good is the model with train data?
  y_pred_[i],p_pred_[i] = prediction(X_sample_[i],coeffs)
  Y_pred_[i] = evaluation(y_sample_[i],p_pred_[i])

1
#################################
Accuracy =  0.8681154611811546
#################################
2
#################################
Accuracy =  0.8699402786994028
#################################
3
#################################
Accuracy =  0.8701061712010617
#################################
4
#################################
Accuracy =  0.8691108161911082
#################################
5
#################################
Accuracy =  0.8772395487723955
#################################
6
#################################
Accuracy =  0.869442601194426
#################################


In [ ]:
#Formating the models for ensemble input
model_list = list(model_.values())
indice_list = list(np.array(list(model_.keys()), dtype=str))
combined_original=[]
for i in range (len(model_)):
  combined_original.append((indice_list[i],model_list[i]))

In [ ]:
#Running ensemble input
ensemble_original = VotingClassifier(estimators=combined_original, voting='soft')
ensemble_original.fit(X_train,y_train)

VotingClassifier(estimators=[('1',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg')),
                             ('2',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg')),
                             ('3',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg')),
                             ('4',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg')),
                             ('5',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg')),
                             ('6',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg'))],
                 voting='soft')

In [ ]:
y_pred_original = ensemble_original.predict(X_train)
p_pred_original = ensemble_original.predict_proba(X_train)[:,1]
Y_pred_original = evaluation(y_train,p_pred_original)

#################################
Accuracy =  0.8961236452112364
#################################


In [ ]:
found = False

ensemble_={}
ensemblefit_={}

X_train_minus_={}
y_train_minus_={}

# Iterate over each model
for i in range(1,n+1,1):
  model_copy = model_.copy()
  X_sample_copy = X_sample_[i].iloc[:, 0:43].copy()
  y_sample_copy = y_sample_[i].copy()
  model_copy.pop(i)

  # Remove X_sample_copy out of X_train
  X_train_minus_[i] = pd.concat([X_train, X_sample_copy]).drop_duplicates(keep=False)

  # Remove y_sample_copy out of y_train
  y_train_minus_[i] = y_train.drop(list(y_sample_copy.index))

  model_list = list(model_copy.values())
  indice_list = list(np.array(list(model_copy.keys()), dtype=str))
  combined=[]
  for j in range (len(model_copy)):
      combined.append((indice_list[j],model_list[j]))


  #Running ensemble input
  ensemble_[i] = VotingClassifier(estimators=combined, voting='soft', n_jobs=-1)
  ensemble_[i].fit(X_train_minus_[i],y_train_minus_[i])



In [ ]:
ensemble_[n+1] = VotingClassifier(estimators=combined_original, voting='soft')
ensemble_[n+1].fit(X_train,y_train)

VotingClassifier(estimators=[('1',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg')),
                             ('2',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg')),
                             ('3',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg')),
                             ('4',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg')),
                             ('5',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg')),
                             ('6',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000, random_state=42,
                                                 solver='newton-cg'))],
                 voting='soft')

In [ ]:
def input_perturbation(X, S, epsilon, delta):
  '''
    X: training dataset
    S: a d-dimensional vector that holds sensitivity values of each feature in X_i (training example).
    epsilon: privacy budget
    delta: failure probability (delta = 0 for pure epsilon-DP)
    function returns perturbed version of X.
  '''
  # From Homework 5 before scaling the distributions were way off the original, after scaling probabilties remained close
  d=len(S)
  X_perturbed = X.copy()
  for i in range(len(X)):
    for j in range(d):
        if j < 9: #ignoring numerical columns
          X_perturbed[i][j]=X[i][j]
        else:
          upper_bound, lower_bound = S[j]
          noise = np.random.normal(loc=0, scale=(upper_bound-lower_bound) * np.sqrt(2*np.log(1.25/delta))/ (epsilon*d))
          X_perturbed[i][j] = X[i][j] + noise
  return X_perturbed
#Function to determine the sensitivity for each feature of X_train
def sensitivity_vector(X):
    sensitivity_vec = []
    for j in range(X.shape[1]):
        sensitivity = (np.max(X[:, j]), np.min(X[:, j]))
        sensitivity_vec.append(sensitivity)
    return sensitivity_vec

In [ ]:
delta = 1e-5
epsilon = 0.8

In [ ]:
X_attack_col = X_attack.columns

In [ ]:
X_attack_vector= X_attack.values

In [ ]:
S= sensitivity_vector(X_attack_vector)

In [ ]:
X_attack_perturbed = input_perturbation(X_attack_vector, S, epsilon, delta)

In [ ]:
X_attack_df=pd.DataFrame(X_attack_perturbed, columns=X_attack_col)

In [ ]:
columns_to_round = X_attack_df.columns[8:43]
means = X[columns_to_round].mean()
means_df = pd.DataFrame(means, columns=['Mean'])
means_dict = means.to_dict()

In [ ]:
means

In [ ]:


# Function to round values based on condition similarities don't seem to be generated if we hash decimals on the categorical variables
def round_values(val, mean):
    return 0 if val < mean else 1

# Assuming means_dict contains column names as keys and their means as values
for column in columns_to_round:
    mean = means_dict[column]  # Retrieve mean for the column
    X_attack_df[column] = X_attack_df[column].apply(lambda x: round_values(x, mean))


In [ ]:
from datasketch import MinHash, MinHashLSHForest

# Create MinHash objects for each data point
def generate_minhash_objects(data_points, num_perm=128, seed=54):
  minhash_objects = []
  for index, row in data_points.iterrows():
    minhash = MinHash(num_perm=128)  #
    for value in row:
        minhash.update(str(value).encode('utf8'))
    minhash_objects.append(minhash)
  return minhash_objects

In [ ]:
columns_to_convert = X_attack_df.columns[8:]
X_attack_df_hash= X_attack_df.copy()
X_attack_df_hash[columns_to_convert] = X_attack_df_hash[columns_to_convert].astype(np.uint8)

In [ ]:
minhash_objects_X = generate_minhash_objects(X_scaled, num_perm=128)

In [ ]:
forest = MinHashLSHForest(num_perm=128)  # Same num_perm as used for MinHash

# Add MinHash objects to the forest
for i, minhash in enumerate(minhash_objects_X):
    forest.add(i, minhash)

# Index the forest
forest.index()

In [ ]:
# Generate MinHash objects for the attack data
minhash_objects_sampled = generate_minhash_objects(X_attack_df_hash, num_perm=128)

In [ ]:
import random

# List to store the indices of the queries
queries_indices = []

# Randomly select 5 indices from minhash_objects_sampled
random_indices = random.sample(range(len(minhash_objects_sampled)), 5000)

for idx in random_indices:
    query_minhash = minhash_objects_sampled[idx]
    result = forest.query(query_minhash, 25)  # Retrieve 25 approximate nearest neighbors

    min_distances = []  # List to store distances for this query
    for neighbor_idx in result:
        distance = 1 - query_minhash.jaccard(minhash_objects_X[neighbor_idx])
        min_distances.append((distance, neighbor_idx))

    # Sort distances and retrieve the nearest 5 neighbors
    min_distances.sort()  # Sort based on distances
    nearest_5 = min_distances[:5]  # Retrieve the nearest 5 neighbors

    # Append the idx value to the list
    queries_indices.append((idx, nearest_5))

# Print the list of indices and nearest 5 neighbors
for query_idx, nearest_5 in queries_indices:
    print(f"Nearest 5 neighbors for Query {query_idx}: {nearest_5}")


Nearest 5 neighbors for Query 4856: [(0.0, 4856), (0.53125, 4886), (0.5390625, 37076), (0.5625, 30438), (0.59375, 27147)]
Nearest 5 neighbors for Query 28753: [(0.0, 28753), (0.4921875, 28829), (0.546875, 34150), (0.5546875, 5625), (0.5546875, 26646)]
Nearest 5 neighbors for Query 15705: [(0.0, 15705), (0.4921875, 37735), (0.5078125, 27329), (0.53125, 24063), (0.5390625, 27040)]
Nearest 5 neighbors for Query 7630: [(0.0, 7630), (0.4609375, 42078), (0.4765625, 7607), (0.4765625, 43603), (0.4765625, 44044)]
Nearest 5 neighbors for Query 19980: [(0.0, 19980), (0.3984375, 1589), (0.421875, 20009), (0.4453125, 35925), (0.53125, 1185)]
Nearest 5 neighbors for Query 37177: [(0.0, 37177), (0.515625, 2705), (0.609375, 40946), (0.6171875, 2562), (0.6171875, 42271)]
Nearest 5 neighbors for Query 18885: [(0.0, 18885), (0.5078125, 18919), (0.5625, 18887), (0.609375, 30108), (0.65625, 18897)]
Nearest 5 neighbors for Query 18818: [(0.0, 18818), (0.5, 11920), (0.53125, 11970), (0.53125, 26722), (0.554

In [ ]:
#This is the Actual comparison part of the oracle we moved it out of a function to get it working
queries_indices = []  # List to store the indices of the queries and their nearest neighbors

for idx, query_minhash in enumerate(minhash_objects_sampled):
    result = forest.query(query_minhash, 25)  # Retrieve 25 approximate nearest neighbors

    min_distance = float('inf')  # Initialize minimum distance
    nearest_neighbor_idx = -1  # Initialize index of the nearest neighbor
    # Find the nearest single neighbor in the original dataset from the 25 neighbors
    for neighbor_idx in result:
        distance = 1 - query_minhash.jaccard(minhash_objects_X[neighbor_idx])
        if distance < min_distance:
            min_distance = distance
            nearest_neighbor_idx = neighbor_idx

    queries_indices.append((idx, nearest_neighbor_idx))  # Store the index of the nearest neighbor

In [ ]:
#Approximate SIGNATURE MATCHING EXCLUSION - this is just the exclusion part of the oracle
def AESOracle(user_input,ensemble_,X_sample_):

  found = False

  # Iterate over each model
  for i in range(1, n + 1):
      numerical_indexes = X_sample_[i].index.tolist()
      for j in numerical_indexes:
        if user_input[1] == j:
                idx = X_sample_[i].index.get_loc(user_input[1])
                return i, idx  # return the model to be excluded and the index of the value in our training set to exclude
  idx = X_scaled.index.get_loc(user_input[1])
  return n + 1, idx # case where no model is to be excluded and the index of the value in our set to use


In [ ]:
# split data into training and test sets: scikit-learn
X_attack_train, X_attack_test, y_attack_train, y_attack_test = train_test_split(queries_indices, y, test_size=0.15)


In [ ]:
print(len(X_attack_train))
print(len(y_attack_train))
print(len(X_attack_test))
print(len(y_attack_test))

38429
38429
6782
6782


In [ ]:
federated_data1 = []

input = X_attack_train

In [ ]:
for row in input:
  federated_data1.append(AESOracle(row,ensemble_,X_sample_))

In [ ]:
federated_data2 = []

input = X_attack_test

for row in input:
  federated_data2.append(AESOracle(row,ensemble_,X_sample_))

In [ ]:
X_attack_train_= {}
X_attack_test_= {}

y_attack_train_={}
y_attack_test_={}

for i in range(1, len(ensemble_)+1):
  X_attack_train_[i] = pd.DataFrame(columns=X_attack_df.columns)
  X_attack_test_[i] = pd.DataFrame(columns=X_attack_df.columns)

  y_attack_train_[i] = pd.Series()
  y_attack_test_[i] = pd.Series()

for modelnumber, idx in federated_data1:
  X_attack_train_[modelnumber] = X_attack_train_[modelnumber].append(X_attack_df.loc[[idx]])
  y_attack_train_[modelnumber] = y_attack_train_[modelnumber].append(y.loc[[idx]])

for modelnumber, idx in federated_data2:
  X_attack_test_[modelnumber] = X_attack_test_[modelnumber].append(X_attack_df.loc[[idx]])
  y_attack_test_[modelnumber] = y_attack_test_[modelnumber].append(y.loc[[idx]])



In [ ]:
X_train_minus_attack_={}
y_train_minus_attack_={}

X_test_minus_attack_={}
y_test_minus_attack_={}

for i in range(1,n+1,1):

  # Remove X_sample_copy_a out of X_train
  X_train_minus_attack_[i] = pd.concat([X_attack_df.iloc[:,0:43], X_attack_train_[i]]).drop_duplicates(keep=False)

  # Remove y_sample_copy_a out of y_train
  y_train_minus_attack_[i] = y.drop(list(y_attack_train_[i].index))


  # Remove X_sample_copy_a out of X_train
  X_test_minus_attack_[i] = pd.concat([X_attack_df.iloc[:,0:43], X_attack_test_[i]]).drop_duplicates(keep=False)

  # Remove y_sample_copy_a out of y_train
  y_test_minus_attack_[i] = y.drop(list(y_attack_test_[i].index))


In [ ]:
X_train_minus_attack_[7] = pd.concat([X_attack_df.iloc[:,0:43], X_attack_train_[7]]).drop_duplicates(keep=False)
X_test_minus_attack_[7] = pd.concat([X_attack_df.iloc[:,0:43], X_attack_test_[7]]).drop_duplicates(keep=False)
y_train_minus_attack_[7] = y.drop(list(y_attack_train_[7].index))
y_test_minus_attack_[7] = y.drop(list(y_attack_test_[7].index))

In [ ]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBoxRuleBased
from art.estimators.classification import SklearnClassifier
#model
#ensemble_original

for i in range(1, len(ensemble_)+1):
  # Step 1: Choose a classifier
  classifier = ensemble_[i]

  if i != len(ensemble_):
    # Step 2: Train the classifier
    classifier.fit(X_train_minus_[i].to_numpy(), y_train_minus_[i].to_numpy())
  else:
    classifier.fit(X_test.to_numpy(), y_test.to_numpy())

  # Step 3: Convert the scikit-learn classifier to an ART-compatible classifier
  estimator = SklearnClassifier(model=classifier)

  # Create the MembershipInferenceBlackBoxRuleBased attack
  attack = MembershipInferenceBlackBoxRuleBased(estimator)

  # Now you can proceed with the attack using the specified classifier
  import numpy as np
  from art.attacks.inference.membership_inference import MembershipInferenceBlackBoxRuleBased

  # infer attacked feature
  inferred_train = attack.infer(X_train_minus_attack_[i].to_numpy(), y_train_minus_attack_[i].to_numpy())
  inferred_test = attack.infer(X_test_minus_attack_[i].to_numpy(), y_test_minus_attack_[i].to_numpy())

  # check accuracy
  train_acc = np.sum(inferred_train) / len(inferred_train)
  test_acc = 1 - (np.sum(inferred_test) / len(inferred_test))
  acc = (train_acc * len(inferred_train) + test_acc * len(inferred_test)) / (len(inferred_train) + len(inferred_test))
  print(f"Members Accuracy: {train_acc:.4f}")
  print(f"Non Members Accuracy {test_acc:.4f}")
  print(f"Attack Accuracy {acc:.4f}")

Members Accuracy: 0.4923
Non Members Accuracy 0.4865
Attack Accuracy 0.4892
Members Accuracy: 0.5163
Non Members Accuracy 0.4629
Attack Accuracy 0.4883
Members Accuracy: 0.5647
Non Members Accuracy 0.4154
Attack Accuracy 0.4864
Members Accuracy: 0.4610
Non Members Accuracy 0.5184
Attack Accuracy 0.4911
Members Accuracy: 0.5484
Non Members Accuracy 0.4303
Attack Accuracy 0.4864
Members Accuracy: 0.5356
Non Members Accuracy 0.4434
Attack Accuracy 0.4872
Members Accuracy: 0.4428
Non Members Accuracy 0.5565
Attack Accuracy 0.5041


In [ ]:
#attack data was stored differently than the ESE oracle, to generate some graphs on the unprotected data, had to reconstruct it...
attack_train_indexes = X_attack_train
attack_test_indexes = X_attack_test

X_attack_train2 = []
X_attack_test2 = []

# For X_attack_train
for index_tuple in attack_train_indexes:
    index = int(index_tuple[1])
    row = X_attack_df.iloc[index]
    X_attack_train2.append(row)

# For X_attack_test
for index_tuple in attack_test_indexes:
    index = int(index_tuple[1])
    row = X_attack_df.iloc[index]
    X_attack_test2.append(row)


In [ ]:
X_attack_train_df = pd.DataFrame(X_attack_train2)
X_attack_test_df = pd.DataFrame(X_attack_test2)

In [ ]:
classifier = model
classifier.fit(X_train.to_numpy(), y_train.to_numpy())
estimator = SklearnClassifier(model=classifier)
attack = MembershipInferenceBlackBoxRuleBased(estimator)

In [ ]:
# infer attacked feature
from art.attacks.inference.membership_inference import MembershipInferenceBlackBoxRuleBased
from art.estimators.classification import SklearnClassifier
inferred_train = attack.infer(X_attack_train_df.to_numpy(), y_attack_train.to_numpy())
inferred_test = attack.infer(X_attack_test_df.to_numpy(), y_attack_test.to_numpy())

# check accuracy
train_acc = np.sum(inferred_train) / len(inferred_train)
test_acc = 1 - (np.sum(inferred_test) / len(inferred_test))
acc = (train_acc * len(inferred_train) + test_acc * len(inferred_test)) / (len(inferred_train) + len(inferred_test))
print(f"Members Accuracy: {train_acc:.4f}")
print(f"Non Members Accuracy {test_acc:.4f}")
print(f"Attack Accuracy {acc:.4f}")

Members Accuracy: 0.5469
Non Members Accuracy 0.4493
Attack Accuracy 0.5322


In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 1431.6762282848358 seconds ---
